In [ ]:
import tables
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
f = tables.open_file("gamma_cone10_sample.h5", "r")

### Provenance information stored as attributes

* Versions of involved code
* Corsika and simtel main parameters
* Simtel filenames

In [ ]:
f.root._v_attrs

### Flat file structure: everything else is stored as tables

* Three tables containing:
    + Array information
    + Telescope type information
    + Event information
* Plus one table per telescope containing:
    + 2x 1D arrays with calibrated pixels data per image (charge, arrival time)

In [ ]:
for node in f:
    print(node)

### Array information table

In [ ]:
f.root.Array_Info

In [ ]:
tel_types=['LST','MSTS','SSTC']

arr_table=f.root.Array_Info

fig, ax = plt.subplots(figsize=(10,10))

for tel_type in tel_types:
    tel_x = [x['tel_x'] for x in arr_table.iterrows() if x['tel_type'] == tel_type.encode('ascii')]
    tel_y = [x['tel_y'] for x in arr_table.iterrows() if x['tel_type'] == tel_type.encode('ascii')]
    plt.scatter(tel_x, tel_y, label=tel_type)
    
ax.legend()
ax.grid()
plt.show()

### Telescope information table

In [ ]:
f.root.Telescope_Info

### Event information table

In [ ]:
f.root.Event_Info

### Example 1: extract some infos from a given event

In [ ]:
event_index = 44
my_event = f.root.Event_Info[event_index]
print('Event number: {}'.format(my_event['event_number']))
print('Energy: {} TeV'.format(my_event['mc_energy']))
print('Alt: {} rad'.format(my_event['alt']))
print('Az: {} rad'.format(my_event['az']))
tel_types = ['LST','MSTS','SSTC']
for tel_type in tel_types:
    tel_indices = '{}_indices'.format(tel_type)
    print('{} = {}'.format(tel_indices,my_event[tel_indices]))

### Example 2: plot all charge and arrival time images for a given event

In [ ]:
tel_types=['LST','MSTS','MSTN','MSTF','SSTC','SST1','SSTA']

tel_type_size={'LST':15,'MSTS':2,'MSTN':15,'MSTF':15,'SSTC':15,'SSTA':15,'SST1':15} #Size of the scatter plot point, for visualization purposes only

tel_table=f.root.Telescope_Info

for tel_type in tel_types:
    pos_x, pos_y = [x['pixel_pos'] for x in tel_table.iterrows() if x['tel_type'] == tel_type.encode('ascii')][0]
    tel_ids = [x['tel_id'] for x in arr_table.iterrows() if x['tel_type'] == tel_type.encode('ascii')]
    exec('my_images = f.root.{}'.format(tel_type))
    my_indices = my_event['{}_indices'.format(tel_type)]
    for img_index in my_indices:
        if img_index > 0:
            img_charge = my_images[img_index]['image_charge']
            img_time = my_images[img_index]['image_peak_times']
            print('Telescope type: {}, ID: {}'.format(tel_type,tel_ids[list(my_indices).index(img_index)]))
            plt.figure(figsize=(15,5))
            plt.subplot(1, 2, 1)
            plt.scatter(pos_x[:len(img_charge)], pos_y[:len(img_charge)], c=img_charge, s=tel_type_size[tel_type])
            plt.axis('square')
            plt.colorbar()
            plt.subplot(1, 2, 2)
            plt.scatter(pos_x[:len(img_time)], pos_y[:len(img_time)], c=img_time, s=tel_type_size[tel_type])
            plt.axis('square')
            plt.colorbar()
            plt.show()